<!-- %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% Calculate a theoretical OTF for a structured illumination microscope %
% James Manton, 2019
% Modified by Antone Bajor 2022 to generate synthetic modulated psf and for
% Synthetic data for SIM Reconstruction
% Pupil calculations http://kmdouglass.github.io/posts/simple-pupil-function-calculations/
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% -->

<!-- Originally Matlab Code refactored for python by Antone Bajor 2023 -->
<!-- The Orignial Matlab Code is much faster -->

In [1]:
import numpy as np
from numpy import fft
from numpy import matlib
#import cupy as cp
import tifffile as tff
import math
import deconvtools as dt # https://github.com/AllenInstitute/render-python-apps/blob/master/renderapps/intensity_correction/deconvtools.py

Some configuration flags etc.

In [2]:
INTERFEROMETRIC_DETECTION = False
SAVE_IMAGES = True
SAVE_OTF = True
SAVE_PSF = True
root_name = '3DSIM'



%% Simulation parameters
%% Values for our system
%% Odd value gives a center point and center planes.
%% If Odd value isn't used modulation field will have a diagonal drift

In [3]:
field_size = 513 #The 3D voxel count NxNxN                                                                                                                                                                                        ;
numerical_aperture_primary = 1.3 #1.3; %% Numerical Aperture of the Objective
numerical_aperture_detection = numerical_aperture_primary
numerical_aperture_secondary = numerical_aperture_primary
refractive_index = 1.4 #1.518;%1.4; %% Refractive Index of the immersion oil

%% For Generating PSF and OTF use emmision wavelength
%% For Generating Modulation Field use excitation wavelength

In [16]:
wave_length = 488e-9 #525e-9;
slm_pixel_size = 9.2e-6 # %% Structured Light Modulator Pixel size
pixels_per_period = 14 # %% Period of pattern
f_tubelens = 180e-3 # %% Focal Length of Tube Lens
mag_obj = 60        # %% Magnification of Tube Lens
f_objective = f_tubelens/mag_obj # %% Focal Length of Objective

f_slm_lens = 800e-3 #610e-3;%500e-3;%610e-3; %% Focal Length of the SLM

rd = f_objective * numerical_aperture_primary # %% Pupil Radius

#%% Difraction angle of 1st orders
theta_1st = math.asin(wave_length/(slm_pixel_size*pixels_per_period))
r_1st = np.tan(theta_1st)*f_slm_lens # %% Radius of 1st orders in pupil

%% For best performance pixel pitch will need to be compared with the ability to laterally shift simulated modulation period by desired amount, jumps in shift will cause funny seperation bands, due to the fact that the 3D SIM seperation matrix expects equidistant phase shifts.

In [21]:
pix_pitch = 30e-9 # %% Sampling of pixels (use at most 1/2 actual camera sensor for simulation)
wl_str = "{:.0f}".format(wave_length*1e9) + 'nm_pix_pitch_' + "{:.0f}".format(pix_pitch*1e9) + 'nm_pix field_' + str(field_size)
freq_NA = numerical_aperture_primary/wave_length #High frequency supported by system 1/m
freq_NIMM = refractive_index/wave_length
freq_samp = 1/pix_pitch #sampling frequency
dFreq = freq_samp/(field_size) # 1/(m*pix)
pupilRad = (freq_NA/dFreq) # pupil radius in pixels
sphereRad = freq_NIMM/dFreq
pupil_per_rad = pupilRad/((field_size)/2) # % percent of 1/2 field radius
prim_a_r = math.asin(numerical_aperture_primary/refractive_index) # % half angle alpha
sphere_rad = pupilRad/np.sin(prim_a_r)
sphere_per_rad = sphere_rad/((field_size - 1)/2) # % Radius of sphere that defines spherical cap
spot_radius_per = r_1st / rd # % 1st order spot radius in percentage of pupil radius
spot_radius = pupilRad*spot_radius_per
pupHeight = sphere_rad*np.cos(prim_a_r) # %% Height to the bottom of spherical cap
height_z = sphere_rad - pupHeight # %% Height of the spherical cap
theta = np.arccos(spot_radius/sphere_rad)
spot_height = spot_radius*np.tan(theta)
zero_height = sphere_rad - spot_height
freq_kz = zero_height*dFreq
freq_kxy = spot_radius*dFreq

print('freq_NA: ' + str(freq_NA) + ', freq_samp: ' + str(freq_samp) + ', dFreq: ' + str(dFreq))
print('pupilRad: ' + str(pupilRad))
print('sphereRad_NIMM: ' + str(sphereRad))
print('pupil_per_rad: ' + str(pupil_per_rad))
print('half angle alpha: ' + str(prim_a_r*180/math.pi))
print('should be same: ' + str(math.asin(pupilRad/sphereRad)*180/math.pi))
print('sphere_rad: ' + str(sphere_rad))
print('sphere_per_rad: ' + str(sphere_per_rad))
print('spot_radius: ' + str(pupilRad*spot_radius))
print('spot_radius per: ' + str(spot_radius))
print('pupHeight: ' + str(pupHeight))
print('height_z: ' + str(height_z))
print('********** IMPORTATN **********') # Meme game too stronk
print('freq_kz: ' + str(freq_kz) + ', freq_kxy: ' + str(freq_kxy))
print('course period kz: ' + str(1/freq_kz*1e9) + 'nm, course period kxy: ' + str(1/freq_kxy*1e9) + 'nm')
print('*******************************')


freq_NA: 2663934.426229508, freq_samp: 33333333.333333336, dFreq: 64977.25795971411
pupilRad: 40.99795081967213
sphereRad_NIMM: 44.15163934426228
pupil_per_rad: 0.15983606557377047
half angle alpha: 68.21321070173822
should be same: 68.21321070173822
sphere_rad: 44.151639344262286
sphere_per_rad: 0.17246734118852455
spot_radius: 1306.3415981720761
spot_radius per: 31.863582741439156
pupHeight: 16.387046267511266
height_z: 27.76459307675102
********** IMPORTATN **********
freq_kz: 882971.2514009865, freq_kxy: 2070408.2353111864
course period kz: 1132.5397043372898nm, course period kxy: 482.9965332173721nm
*******************************


Create Coordinates for voxel volume

In [6]:

lim = field_size - (field_size + 1)/2
x = np.arange(-lim,lim + 1, 1, dtype=np.float32)
y = np.arange(-lim,lim + 1, 1, dtype=np.float32)
z = np.arange(-lim,lim + 1, 1, dtype=np.float32)
[X, Y, Z] = np.meshgrid(x, y, z, indexing='ij')
R = (np.sqrt(X**2 + Y**2 + Z**2)).astype(np.float32)
phi = (np.arctan2(X, Y) * 180 / np.pi).astype(np.float32)
theta = (np.arctan2(X, Z) * 180 / np.pi).astype(np.float32)

# %% this gives a 3D cylindrical column representing percentage of radius
R = R / np.max(x)
r = np.sqrt(X**2 + Y**2)

# %% This gives a radius of the field as a percentage
r = r / np.max(x)

In [7]:
print('x.shape: ' + str(x.shape))
print('r.shape: ' + str(r.shape))
print('R.shape: ' + str(R.shape))
print('Z.shape: ' + str(Z.shape))
print('Y.shape: ' + str(Y.shape))
print(R.size)

x.shape: (513,)
r.shape: (513, 513, 513)
R.shape: (513, 513, 513)
Z.shape: (513, 513, 513)
Y.shape: (513, 513, 513)
135005697


Build the Pupil

In [8]:
# % Detection OTF
r = np.swapaxes(r, 0, 2)
detection_pupil = np.zeros((field_size, field_size, field_size)).astype(np.float32)
detection_pupil[r < pupil_per_rad] = 1 # %% generates the detection pupil
tff.imwrite('detection_pupil_' + wl_str + str(pixels_per_period) + 'pix_3D.tif', np.abs(detection_pupil).astype(np.float32))
detection_ctf = np.zeros((field_size, field_size, field_size)).astype(np.complex64)
R = np.ravel(np.swapaxes(R, 0, 2))
Z = np.ravel(np.swapaxes(Z, 0, 2))
detection_pupil = np.ravel(detection_pupil)
detection_ctf = np.ravel(detection_ctf)

# %% The Else Statement generates the sphereical cap, not using interferometric detection
if INTERFEROMETRIC_DETECTION:
    detection_ctf[R < (sphere_per_rad) & R > (sphere_per_rad - 0.01) & detection_pupil > 0] = 1
else:
    mask = (R < (sphere_per_rad + 0.01)) & (R > (sphere_per_rad)) & (detection_pupil > 0) &  (Z < 0)
    detection_ctf[mask == True] = 1
print('past ravel loop')
detection_ctf = np.reshape(detection_ctf, [field_size, field_size, field_size])

#%detection_ctf = imgaussfilt3(detection_ctf);
tff.imwrite('detection_ctf' + wl_str + str(pixels_per_period) + 'pix_3D.tif', np.abs(detection_ctf).astype(np.float32))

#%% This generates the OTF auto corelation
detection_otf = (fft.fftshift(fft.ifftn(fft.fftn(detection_ctf) * np.conj(fft.fftn(detection_ctf)))))

if (SAVE_OTF):
    detection_otf[detection_otf < 0] = 0
    tff.imwrite('detection' + '_otf_' + 'float_' +  wl_str + str(pixels_per_period) + 'pix_3D.tif', np.abs(detection_otf).astype(np.float32))

past ravel loop


This is where the modulation field is generated.

In [9]:
# %% Primary OTF

primary_pupil = np.zeros((field_size, field_size, field_size)).astype(np.float32)
# primary_pupil = np.swapaxes(primary_pupil, 1, 2)

# %% For some reason this part of the code uses the 1/2 (Field - 1) value for radius instead of r array
# %% This first part of the code generates the 1st order spots extruded through z axis

Y = (np.swapaxes(Y, 0, 2))
X = (np.swapaxes(X, 0, 2))

Y = np.ravel(np.swapaxes(Y,2, 1))
X = np.ravel(np.swapaxes(X, 2, 1))
# x = np.swapaxes(x, 0, 1)
primary_pupil = np.ravel(primary_pupil)

mask = (np.floor(abs(Y)) == np.round(np.max(x) * spot_radius * pupil_per_rad)) & (np.floor(X) == 0)
primary_pupil[mask] = 1

#%% This second part generates the 0th order spot extruded thorough z axis;
primary_pupil[(np.floor(Y) == 0) & (np.floor(X) == 0)] = 1

primary_ctf = np.zeros((field_size,field_size,field_size)).astype(np.complex64)
primary_ctf = np.ravel(primary_ctf)

#%% This takes the 3 beam orders "primary_pupil" and places them only where they intersect
#%% Along the spherical shell using the pupil radius is incorrect since it defines a smaller
#%% radius sphere, which gives incorrect axial modulation.
primary_ctf[(R < (sphere_per_rad + 0.01)) & (R > (sphere_per_rad)) & (primary_pupil > 0) & (Z > 0)] = 1
primary_ctf = np.reshape(primary_ctf, (field_size, field_size, field_size))
primary_pupil = np.reshape(primary_pupil, (field_size, field_size, field_size))
tff.imwrite('primary_pupil' + '_3D.tif', primary_pupil.astype(np.float32))

# % Fix band weights
primary_ctf_sum = np.repeat(matlib.repmat(np.sum(primary_ctf, 0), 1, 1)[np.newaxis,:,:], field_size, axis=0) # %% not sure this gives different value than primary_ctf in my application
print(primary_ctf_sum.shape)
tff.imwrite('primary_repmat' + '_3D.tif', np.abs(primary_ctf_sum).astype(np.float32))
primary_ctf[primary_ctf > 0] = primary_ctf[primary_ctf > 0] / primary_ctf_sum[primary_ctf > 0]

tff.imwrite('primary_ctf_' + wl_str + '_3D.tif',np.abs(primary_ctf).astype(np.float32))

# %% This step generates the 7 beam spots that can been seen in the xz center plane
# %% This defines the 3D sim intensity modulation field
primary_otf = (fft.fftshift(fft.ifftn(fft.fftn(primary_ctf) * np.conj(fft.fftn(primary_ctf)))))
tff.imwrite('modulation_otf_' + wl_str + '_3D.tif', np.abs(primary_otf).astype(np.float32))

(513, 513, 513)


In [10]:
# %% Overall OTF

# %% Converts detection otf to psf

detection_psf = fft.fftshift(fft.fftn(detection_otf).astype(np.complex64))


if (SAVE_PSF):
   tff.imwrite('Detection_psf_' + wl_str + 'float' + '_3D.tif', np.abs(detection_psf).astype(np.float32))

#%% Creates the realspace sim Modulation field
modulation_field = dt.otf2psf(primary_otf, primary_otf.shape).astype(np.complex64)
#%% Scales max intensity to 1, and removes checker board patterning taking abs value
modulation_field = np.abs(modulation_field/np.max(modulation_field))

min_modulation = np.min(modulation_field)
print("min_modulation: " + str(min_modulation)) # %% if this is 1 something went wrong
tff.imwrite('modulation_field_' + wl_str + '_' + str(pixels_per_period) + 'pix_3D.tif',modulation_field)


min_modulation: 0.0
